### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.sum(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for i in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0       
        # For each user-movie pair
        for u in range(n_users):
            for m in range(n_movies):
                # if the rating exists
                if ratings_mat[u, m] > 0:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = ratings_mat[u, m] - np.dot(user_mat[u, :], movie_mat[:, m])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error ** 2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[u, k] += learning_rate * 2 * error * movie_mat[k, m]
                        movie_mat[k, m] += learning_rate * 2 * error * user_mat[u, k]
        # print results for iteration
        print('%d \t\t\t %.2f' % (i + 1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 			 42.66
2 			 14.65
3 			 4.08
4 			 2.83
5 			 2.74
6 			 2.71
7 			 2.69
8 			 2.67
9 			 2.64
10 			 2.62


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99067055  8.91974345 10.61511211 10.17680636]
 [ 8.88798437  7.39044009  9.09306223  8.67888644]
 [ 8.13428371  6.6859748   8.35573292  7.65987904]
 [ 9.7276517   8.2036482  10.14723816  9.42835289]
 [ 8.67499486  7.45078353  8.83733187  8.5594235 ]
 [ 6.67424241  6.08562522  7.20251444  6.91228846]
 [ 9.40972     8.11822465  9.40836023  9.27086755]
 [ 8.36820856  7.19876998  8.58441516  8.11650585]
 [ 8.42109649  7.2724367   9.16659059  8.36253007]
 [ 7.79282814  6.78292957  8.00490665  7.62070339]
 [ 8.72698882  7.92108543  9.15428039  9.01051474]
 [ 9.39316737  8.41907653 10.12220796  9.60895852]
 [ 9.4357958   7.86676682  9.89479938  9.15284035]
 [ 6.43195647  6.08542138  6.56561385  6.91392488]
 [ 9.92777973  8.38881963 10.25134903  9.69600599]
 [ 9.86008375  8.51966995 10.19717188  9.69128282]
 [ 8.13529641  7.25493812  8.72790862  8.36112695]
 [ 7.10826049  6.35335534  7.2524119   7.16411049]
 [ 8.17134222  7.27515588  8.89589233  8.43726357]
 [ 7.9867229   7.29040561  8.34

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 			 42.03
2 			 14.45
3 			 4.18
4 			 2.83
5 			 2.66
6 			 2.60
7 			 2.55
8 			 2.50
9 			 2.45
10 			 2.39
11 			 2.32
12 			 2.25
13 			 2.17
14 			 2.09
15 			 2.00
16 			 1.91
17 			 1.82
18 			 1.73
19 			 1.64
20 			 1.55
21 			 1.48
22 			 1.40
23 			 1.33
24 			 1.27
25 			 1.21
26 			 1.15
27 			 1.10
28 			 1.05
29 			 1.00
30 			 0.95
31 			 0.90
32 			 0.85
33 			 0.81
34 			 0.77
35 			 0.74
36 			 0.70
37 			 0.67
38 			 0.64
39 			 0.62
40 			 0.60
41 			 0.58
42 			 0.57
43 			 0.56
44 			 0.55
45 			 0.54
46 			 0.53
47 			 0.52
48 			 0.52
49 			 0.52
50 			 0.51
51 			 0.51
52 			 0.51
53 			 0.50
54 			 0.50
55 			 0.50
56 			 0.50
57 			 0.50
58 			 0.50
59 			 0.50
60 			 0.49
61 			 0.49
62 			 0.49
63 			 0.49
64 			 0.49
65 			 0.48
66 			 0.48
67 			 0.48
68 			 0.48
69 			 0.47
70 			 0.47
71 			 0.47
72 			 0.46
73 			 0.46
74 			 0.45
75 			 0.45
76 			 0.44
77 			 0.44
78 			 0.43
79 			 0.42
80

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99996065  9.99998447  9.99999387 10.00003523]
 [ 9.999991    3.99999742  8.99999939 10.00000782]
 [ 8.00005108  9.00002398 10.00001066  4.99995202]
 [ 9.00008348  8.000037   10.00001567  9.99992225]
 [10.000011    5.00000468  9.00000154  8.99998882]
 [ 6.00001218  4.00000526 10.0000018   5.99998759]
 [ 9.00000743  8.00000302 10.00000067  8.99999173]
 [10.00009405  5.00004002  9.00001606  7.9999122 ]
 [ 7.00019659  8.00008322 10.00003347  7.9998172 ]
 [ 9.00009564  5.00003784  9.00001334  6.9999096 ]
 [ 8.99998017  7.99998711  9.99999133  8.00001513]
 [ 8.99988018  9.99994472  9.99997398  9.00010729]
 [10.00003659  9.00001378 10.00000394  7.99996399]
 [ 5.00013758  8.00005793  5.00002315  7.99987211]
 [ 9.99987258  7.99994155  9.99997282 10.00011454]
 [ 9.0000701   9.00002837 10.00001019  9.99993331]
 [ 8.9998172   7.99991958  7.99996487  8.00016636]
 [10.00002505  8.00001113  1.00000479  9.99997684]
 [ 4.99991902  5.99996422  9.999984   10.00007293]
 [ 7.99996659  6.99998641  9.99

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 			 43.01
2 			 14.72
3 			 4.21
4 			 2.91
5 			 2.77
6 			 2.74
7 			 2.72
8 			 2.69
9 			 2.66
10 			 2.63
11 			 2.59
12 			 2.55
13 			 2.50
14 			 2.45
15 			 2.39
16 			 2.32
17 			 2.24
18 			 2.15
19 			 2.06
20 			 1.96
21 			 1.86
22 			 1.75
23 			 1.65
24 			 1.55
25 			 1.46
26 			 1.37
27 			 1.29
28 			 1.21
29 			 1.15
30 			 1.08
31 			 1.02
32 			 0.97
33 			 0.92
34 			 0.86
35 			 0.81
36 			 0.76
37 			 0.71
38 			 0.66
39 			 0.61
40 			 0.56
41 			 0.51
42 			 0.46
43 			 0.41
44 			 0.37
45 			 0.33
46 			 0.29
47 			 0.25
48 			 0.22
49 			 0.19
50 			 0.17
51 			 0.14
52 			 0.12
53 			 0.11
54 			 0.09
55 			 0.08
56 			 0.07
57 			 0.06
58 			 0.05
59 			 0.04
60 			 0.03
61 			 0.03
62 			 0.03
63 			 0.02
64 			 0.02
65 			 0.02
66 			 0.01
67 			 0.01
68 			 0.01
69 			 0.01
70 			 0.01
71 			 0.01
72 			 0.01
73 			 0.00
74 			 0.00
75 			 0.00
76 			 0.00
77 			 0.00
78 			 0.00
79 			 0.00
80

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.18596802068854:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 			 23.30
2 			 10.72
3 			 7.36
4 			 5.68
5 			 4.62
6 			 3.87
7 			 3.32
8 			 2.90
9 			 2.56
10 			 2.28
11 			 2.06
12 			 1.87
13 			 1.71
14 			 1.58
15 			 1.46
16 			 1.37
17 			 1.28
18 			 1.21
19 			 1.14
20 			 1.08


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [11]:
np.sum(~np.isnan(first_1000_users))

10852

In [12]:
np.sum(np.isnan(first_1000_users))

31234148

In [13]:
# Replace each of the comments below with the correct values
num_ratings = 10852 # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = 31234148 # How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [14]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
